# Opdracht 3.3: Race Conditions

Race conditions kunnen optreden als twee of meer threads draaien die een stuk data delen. Race conditions zijn de problemen die optreden als meerdere processen/threads tegelijkertijd variabelen proberen te (over)schrijven.  
  
In veel gevallen treden race conditions maar af-en-toe op, waardoor ze ook zo lastig zijn om te debuggen (het programma doet dan ineens iets wat niet verwacht wordt).

In [1]:
import threading, logging, time
import concurrent.futures

class FakeDatabase:
    def __init__(self):
        self.value = 0

    def update(self, name):
        logging.info("Thread %s: starting update", name)
        local_copy = self.value
        local_copy += 1
        time.sleep(0.1)
        self.value = local_copy
        logging.info("Thread %s: finishing update", name)

In [2]:
if __name__ == "__main__":
    format = "%(asctime)s: %(message)s"
    logging.basicConfig(format=format, level=logging.INFO,
                        datefmt="%H:%M:%S")

    database = FakeDatabase()
    logging.info("Testing update. Starting value is %d.", database.value)
    with concurrent.futures.ThreadPoolExecutor(max_workers=2) as executor:
        for index in range(2):
            executor.submit(database.update, index)
    logging.info("Testing update. Ending value is %d.", database.value)

18:52:23: Testing update. Starting value is 0.
18:52:23: Thread 0: starting update
18:52:23: Thread 1: starting update
18:52:23: Thread 0: finishing update
18:52:23: Thread 1: finishing update
18:52:23: Testing update. Ending value is 1.


In principe zouden we hier verwachten dat het programma, aangezien er twee Threads worden aangemaakt, de value van FakeDatabase uiteindelijk aan het einde opgehoogt heeft tot een waarde van 2.  
  
In dit programma is de Race condition die optreedt de $\textbf{time.sleep(0.1)}$ die intern in de update functie wordt aangeroepen. Hierdoor zullen beide Threads op het moment van starten de waarde van value als een 0 lezen, waardoor ze allebei op het einde de waarde updaten tot een 1.